In [1]:
!pip install mlflow boto3 awscli optuna imbalanced-learn lightgbm

In [2]:
import mlflow
mlflow.set_tracking_uri("http://ec2-18-215-180-118.compute-1.amazonaws.com:5000/")

In [3]:
# Set or create an experiment
mlflow.set_experiment("LightGBM HP Tuning")

2025/07/16 02:28:12 INFO mlflow.tracking.fluent: Experiment with name 'LightGBM HP Tuning' does not exist. Creating a new experiment.


<Experiment: artifact_location='s3://mlflow-bucket-1218/397175165646068793', creation_time=1752611294366, experiment_id='397175165646068793', last_update_time=1752611294366, lifecycle_stage='active', name='LightGBM HP Tuning', tags={}>

In [4]:
import optuna
import mlflow
import mlflow.sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

C:\jupyter\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
df = pd.read_csv('reddit_preprocessing.csv').dropna(subset=['clean_comment'])
df.shape

(36662, 2)

In [6]:
# Step 1: Remap the class labels from [-1, 0, 1] to [2, 0, 1]
df['category'] = df['category'].map({-1: 2, 0: 0, 1: 1})

# Step 2: Remove rows where the target labels (category) are NaN
df = df.dropna(subset=['category'])

In [9]:
# step 3: TF-IDF vectorization setup
ngram_range = (1, 3)  # Trigram 
max_features = 10000  # Set max_features to 1000 for TF-IDF
vectorizer = TfidfVectorizer(ngram_range=ngram_range, max_features=max_features)
X = vectorizer.fit_transform(df['clean_comment'])
y = df['category']

# Step 4: Apply SMOTE to handle class imbalance 
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X , y)

In [12]:
# Step 5: Train-test split 
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X_resampled, y_resampled, 
    test_size=0.2, 
    random_state=42, 
    stratify=y_resampled  
)

In [17]:
# Updated Function to log results in MLflow with optional params and trial number
def log_mlflow(model_name, model, X_train, X_test, y_train, y_test, params=None, trial_num=None):
    with mlflow.start_run():
        # Log model type and run name with trial number (if provided)
        run_name = f"{model_name}_SMOTE_TFIDF_Trigrams"
        if trial_num is not None:
            run_name += f"_Trial_{trial_num}"
        mlflow.set_tag("mlflow.runName", run_name)
        mlflow.set_tag("experiment_type", "algorithm_comparison")

        # Log algorithm name
        mlflow.log_param("algo_name", model_name)

        # Log hyperparameters (if any)
        if params:
            for key, value in params.items():
                mlflow.log_param(key, value)

        # Train the model
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        # Log accuracy
        accuracy = accuracy_score(y_test, y_pred)
        mlflow.log_metric("accuracy", accuracy)

        # Log classification report
        classification_rep = classification_report(y_test, y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)

        # Log the model
        mlflow.sklearn.log_model(model, f"{model_name}_model")

        return accuracy  # Optuna needs this return value

In [18]:
# Step 6: Optuna objective function for LightGBM
def objective_lightgbm(trial):
    # Hyperparameter space to explore
    n_estimators = trial.suggest_int('n_estimators', 100, 1000)
    learning_rate = trial.suggest_float('learning_rate', 1e-4, 1e-1, log=True)
    max_depth = trial.suggest_int('max_depth', 3, 10)
    num_leaves = trial.suggest_int('num_leaves', 20, 150)
    min_child_samples = trial.suggest_int('min_child_samples', 10, 100)
    colsample_bytree = trial.suggest_float('colsample_bytree', 0.5, 1.0)
    subsample = trial.suggest_float('subsample', 0.5, 1.0)
    reg_alpha = trial.suggest_float('reg_alpha', 1e-4, 10.0, log=True)
    reg_lambda = trial.suggest_float('reg_lambda', 1e-4, 10.0, log=True)

    # Log trial parameters
    params = {
        'n_estimators': n_estimators, 
        'learning_rate': learning_rate, 
        'max_depth': max_depth, 
        'num_leaves': num_leaves,
        'min_child_samples':  min_child_samples,
        'colsample_bytree': colsample_bytree,
        'subsample': subsample,
        'reg_alpha': reg_alpha,
        'reg_lambda': reg_lambda
        
    }

    # Create LightGBM model
    model = XGBClassifier(n_estimators=n_estimators,
                          learning_rate=learning_rate,
                          max_depth=max_depth,
                          num_leaves=num_leaves,
                          min_child_samples=min_child_samples,
                          colsample_bytree=colsample_bytree,
                          subsample=subsample,
                          reg_alpha=reg_alpha,
                          reg_lambda=reg_lambda,
                          random_state=42)

    # Log each trial as a separate run in MLflow 
    accuracy = log_mlflow("LightGBM", model, X_train, X_test, y_train, y_test, params, trial.number)
    return accuracy

In [19]:
# Run Optuna for LightGBM log the best model, and plot the importance of each parameter 
def run_optuna_experiment():
    study = optuna.create_study(direction="maximize")
    study.optimize(objective_lightgbm, n_trials=100)     # Increased to 100 trials

    # Get the best parameters
    best_params = study.best_params
    best_model = LGBMClassifier(n_estimators=best_params['n_estimators'],
                          learning_rate=best_params['learning_rate'],
                          max_depth=best_params['max_depth'],
                          num_leaves=best_params['num_leaves'],
                          min_child_samples=best_params['min_child_samples'],
                          colsample_bytree=best_params['colsample_bytree'],
                          subsample=best_params['subsample'],
                          reg_alpha=best_params['reg_alpha'],
                          reg_lambda=best_params['reg_lambda'],
                          random_state=42)
    # Log the best model with MLflow and print the classification report
    log_mlflow("LightGBM", best_model, X_train, X_test, y_train, y_test, best_params, "Best")

    # plot parameter importance
    optuna.visualization.plot_param_importances(study).show()

    # plot optimization history
    optuna.visualization.plot_optimization_history(study).show()

In [20]:
# Run the experiment for LightGBM
run_optuna_experiment()

[I 2025-07-16 02:40:13,526] A new study created in memory with name: no-name-776aa162-2c34-478b-b1de-d47fe9acd9db
C:\jupyter\venv\Lib\site-packages\xgboost\training.py:183: UserWarning: [02:40:21] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "min_child_samples", "num_leaves" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
2025/07/16 02:54:25 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/07/16 02:54:52 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run LightGBM_SMOTE_TFIDF_Trigrams_Trial_0 at: http://ec2-18-215-180-118.compute-1.amazonaws.com:5000/#/experiments/397175165646068793/runs/4cd22ca6f9b84587a13a3f2d9912829b
🧪 View experiment at: http://ec2-18-215-180-118.compute-1.amazonaws.com:5000/#/experiments/397175165646068793


[I 2025-07-16 02:55:17,299] Trial 0 finished with value: 0.6553582752060875 and parameters: {'n_estimators': 637, 'learning_rate': 0.0015707911081195751, 'max_depth': 7, 'num_leaves': 147, 'min_child_samples': 40, 'colsample_bytree': 0.7079028253777275, 'subsample': 0.8338870414219501, 'reg_alpha': 0.00017450964997034487, 'reg_lambda': 0.02149601512648243}. Best is trial 0 with value: 0.6553582752060875.
C:\jupyter\venv\Lib\site-packages\xgboost\training.py:183: UserWarning: [02:55:25] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "min_child_samples", "num_leaves" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
2025/07/16 03:17:01 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/07/16 03:17:25 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run LightGBM_SMOTE_TFIDF_Trigrams_Trial_1 at: http://ec2-18-215-180-118.compute-1.amazonaws.com:5000/#/experiments/397175165646068793/runs/4648d566071e4a028c3108e108fcd9ab
🧪 View experiment at: http://ec2-18-215-180-118.compute-1.amazonaws.com:5000/#/experiments/397175165646068793


[I 2025-07-16 03:17:36,952] Trial 1 finished with value: 0.7866201648700063 and parameters: {'n_estimators': 647, 'learning_rate': 0.015367117213184015, 'max_depth': 10, 'num_leaves': 31, 'min_child_samples': 37, 'colsample_bytree': 0.7905695694448064, 'subsample': 0.8058104922943897, 'reg_alpha': 0.005902815280261613, 'reg_lambda': 5.4236355288482025}. Best is trial 1 with value: 0.7866201648700063.
C:\jupyter\venv\Lib\site-packages\xgboost\training.py:183: UserWarning: [03:17:44] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "min_child_samples", "num_leaves" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
2025/07/16 03:21:28 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/07/16 03:21:48 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


...

🏃 View run LightGBM_SMOTE_TFIDF_Trigrams_Trial_13 at: http://ec2-18-215-180-118.compute-1.amazonaws.com:5000/#/experiments/397175165646068793/runs/f0f7f47d1aaa45acba979741f2900a83
🧪 View experiment at: http://ec2-18-215-180-118.compute-1.amazonaws.com:5000/#/experiments/397175165646068793


[I 2025-07-16 04:54:55,590] Trial 13 finished with value: 0.7950750369900655 and parameters: {'n_estimators': 984, 'learning_rate': 0.02676215681700023, 'max_depth': 4, 'num_leaves': 78, 'min_child_samples': 11, 'colsample_bytree': 0.8096092276630465, 'subsample': 0.9918113606789309, 'reg_alpha': 0.000869521092259799, 'reg_lambda': 0.0007008976283315045}. Best is trial 12 with value: 0.8618685267385331.
C:\jupyter\venv\Lib\site-packages\xgboost\training.py:183: UserWarning: [04:55:03] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "min_child_samples", "num_leaves" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


🏃 View run LightGBM_SMOTE_TFIDF_Trigrams_Trial_14 at: http://ec2-18-215-180-118.compute-1.amazonaws.com:5000/#/experiments/397175165646068793/runs/b5c75e3ab0d84f119fdb4c9d07752f78
🧪 View experiment at: http://ec2-18-215-180-118.compute-1.amazonaws.com:5000/#/experiments/397175165646068793


[W 2025-07-16 05:01:39,896] Trial 14 failed with parameters: {'n_estimators': 835, 'learning_rate': 0.028138210127115152, 'max_depth': 4, 'num_leaves': 49, 'min_child_samples': 24, 'colsample_bytree': 0.9049595184375487, 'subsample': 0.9044445499671364, 'reg_alpha': 7.714336431141125, 'reg_lambda': 0.0020851515066384447} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "C:\jupyter\venv\Lib\site-packages\optuna\study\_optimize.py", line 201, in _run_trial
    value_or_values = func(trial)
  File "C:\Users\latif\AppData\Local\Temp\ipykernel_13356\3259828239.py", line 41, in objective_lightgbm
    accuracy = log_mlflow("LightGBM", model, X_train, X_test, y_train, y_test, params, trial.number)
  File "C:\Users\latif\AppData\Local\Temp\ipykernel_13356\3670938415.py", line 20, in log_mlflow
    model.fit(X_train, y_train)
    ~~~~~~~~~^^^^^^^^^^^^^^^^^^
  File "C:\jupyter\venv\Lib\site-packages\xgboost\core.py", line 729, in inner_f
    return fu

KeyboardInterrupt: 